In [84]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC

In [154]:
# Sacamos todas las rutas de "ver mas" para obtener la mayor cantidad de productos posibles

chrome_options = Options()
chrome_options.add_argument("--disable-popup-blocking")

# Ruta al chromedriver
chromedriver_path = r'C:\Users\pabma\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Inicializa el navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers')
driver.maximize_window()

# Acceptamos las cookies
accept_cokies = driver.find_element(By.ID ,'sp-cc-accept')
accept_cokies.click()
driver.implicitly_wait(10) # Una espera para evitar errores

# a-link-normal
urls_econtradas = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
urls_ver_mas = []
for url in urls_econtradas:
    if url.text == 'Ver más':
        urls_ver_mas.append(url)


In [89]:
for i in urls_ver_mas:
    print(i.get_attribute('href'))
    

https://www.amazon.es/gp/bestsellers/lighting/ref=zg_bs_lighting_sm
https://www.amazon.es/gp/bestsellers/amazon-renewed/ref=zg_bs_amazon-renewed_sm
https://www.amazon.es/gp/bestsellers/hpc/ref=zg_bs_hpc_sm
https://www.amazon.es/gp/bestsellers/kitchen/ref=zg_bs_kitchen_sm
https://www.amazon.es/gp/bestsellers/beauty/ref=zg_bs_beauty_sm
https://www.amazon.es/gp/bestsellers/fashion/ref=zg_bs_fashion_sm


In [155]:
try:   
    clase_link_producto = 'a-link-normal'
    clase_link_hijo = 'aok-block'
    urls_ver_mas[0].click()
    driver.implicitly_wait(10)
    urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)
    driver.implicitly_wait(10)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
    categoria = categoria_en_div.find('h1').text

    lista_df = []
    urls = [elemento.get_attribute('href') for elemento in urls_todos_productos]
    driver.implicitly_wait(10)

    for url_producto in urls:
        # ir = url_producto.get_attribute('href')
        # driver.implicitly_wait(10)

        driver.get(url_producto)
        driver.implicitly_wait(10)

        try:
            clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            reseñas_en_div = soup.find_all('div', class_=clase_producto)
            nombre_producto = soup.find('span', id='productTitle').text

            reseñas = []
            for reseña in reseñas_en_div:
                if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
                    # y otras nos daban nones, por lo que podemos filtrar un poco
                    reseñas.append(reseña.find('span').text)
                    driver.implicitly_wait(10)


            producto = pd.DataFrame()
            producto['reseñas_humanas'] = reseñas
            producto['nombre_producto'] = nombre_producto
            producto['categoria_producto'] = categoria # Esto de momento no cambia
            lista_df.append(producto)
            driver.implicitly_wait(10)

        except Exception as e:
            print(f"Error al extraer información de {url_producto}: {e}") # obsevamos en que url no ha podido entrar

        
        driver.back()

    combined_df = pd.concat(lista_df, ignore_index=True)
except Exception as e:
    print(f"Error al extraer información de {url_producto}: {e}") 





In [157]:
combined_df['reseñas_humanas'].duplicated().sum()

4

In [137]:
# Comprobamos si dentro de una url comprobamos el href de algun producto
clase_link_producto = 'a-link-normal'
clase_link_hijo = 'aok-block'
urls_ver_mas[1].click()
driver.implicitly_wait(10)
urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)
driver.implicitly_wait(10)

print(urls_todos_productos[2].get_attribute('href'))
driver.back()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.56); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6622D1F52+60322]
	(No symbol) [0x00007FF66224CEC9]
	(No symbol) [0x00007FF662107EBA]
	(No symbol) [0x00007FF66211A2CD]
	(No symbol) [0x00007FF66211AD5A]
	(No symbol) [0x00007FF66210ED69]
	(No symbol) [0x00007FF66210CAB6]
	(No symbol) [0x00007FF662110641]
	(No symbol) [0x00007FF6621106E0]
	(No symbol) [0x00007FF66215968C]
	(No symbol) [0x00007FF66214B346]
	(No symbol) [0x00007FF66217C21A]
	(No symbol) [0x00007FF66214ADB6]
	(No symbol) [0x00007FF66217C430]
	(No symbol) [0x00007FF66219BC80]
	(No symbol) [0x00007FF66217BFC3]
	(No symbol) [0x00007FF662149617]
	(No symbol) [0x00007FF66214A211]
	GetHandleVerifier [0x00007FF6625E94AD+3301629]
	GetHandleVerifier [0x00007FF6626

In [98]:
reseñas_en_div[5] # reseña numero 6 del producto en moviles "Apple iPhone 13, 128GB, Blanco estrella - (Reacondicionado)" no tiene sspan o por algun
# motivo no lo ha econtrado
# Da error porque no tiene ninguna reseña, solo valoración en estrellas y una foto

<div aria-expanded="false" class="a-expander-content reviewText review-text-content a-expander-partial-collapse-content" data-hook="review-collapsed"></div>

In [9]:
urls_todos_productos[1].get_attribute('href')

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=126.0.6478.56); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6622D1F52+60322]
	(No symbol) [0x00007FF66224CEC9]
	(No symbol) [0x00007FF662107EBA]
	(No symbol) [0x00007FF66211A2CD]
	(No symbol) [0x00007FF66211AD5A]
	(No symbol) [0x00007FF66210ED69]
	(No symbol) [0x00007FF66211AD98]
	(No symbol) [0x00007FF66210ED69]
	(No symbol) [0x00007FF66210CAB6]
	(No symbol) [0x00007FF662110968]
	(No symbol) [0x00007FF66219C95B]
	(No symbol) [0x00007FF66217C21A]
	(No symbol) [0x00007FF66219BC80]
	(No symbol) [0x00007FF66217BFC3]
	(No symbol) [0x00007FF662149617]
	(No symbol) [0x00007FF66214A211]
	GetHandleVerifier [0x00007FF6625E94AD+3301629]
	GetHandleVerifier [0x00007FF6626336D3+3605283]
	GetHandleVerifier [0x00007FF662629450+3563680]
	GetH